# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Wisconsin', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_3', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'gravity_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 12, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.57182 time= 0.20200
Epoch: 0002 train_loss= 1.30040 time= 0.00400
Epoch: 0003 train_loss= 1.01245 time= 0.00400
Epoch: 0004 train_loss= 0.76592 time= 0.00500
Epoch: 0005 train_loss= 0.86071 time= 0.00400
Epoch: 0006 train_loss= 0.77571 time= 0.00400
Epoch: 0007 train_loss= 0.62254 time= 0.00500
Epoch: 0008 train_loss= 0.57754 time= 0.00301
Epoch: 0009 train_loss= 0.60349 time= 0.00400
Epoch: 0010 train_loss= 0.63595 time= 0.00400
Epoch: 0011 train_loss= 0.62599 time= 0.00300
Epoch: 0012 train_loss= 0.58739 time= 0.00400
Epoch: 0013 train_loss= 0.55340 time= 0.00501
Epoch: 0014 train_loss= 0.52490 time= 0.00315
Epoch: 0015 train_loss= 0.51857 time= 0.00300
Epoch: 0016 train_loss= 0.52427 time= 0.00500
Epoch: 0017 train_loss= 0.53308 time= 0.00300
Epoch: 0018 train_loss= 0.52332 time= 0.00400
Epoch: 0019 train_loss= 0.51182 time= 0

Epoch: 0178 train_loss= 0.38590 time= 0.00400
Epoch: 0179 train_loss= 0.38732 time= 0.00300
Epoch: 0180 train_loss= 0.38462 time= 0.00300
Epoch: 0181 train_loss= 0.38576 time= 0.00400
Epoch: 0182 train_loss= 0.38650 time= 0.00501
Epoch: 0183 train_loss= 0.38662 time= 0.00299
Epoch: 0184 train_loss= 0.38688 time= 0.00400
Epoch: 0185 train_loss= 0.38702 time= 0.00400
Epoch: 0186 train_loss= 0.38525 time= 0.00300
Epoch: 0187 train_loss= 0.38816 time= 0.00401
Epoch: 0188 train_loss= 0.38548 time= 0.00399
Epoch: 0189 train_loss= 0.39161 time= 0.00300
Epoch: 0190 train_loss= 0.38311 time= 0.00400
Epoch: 0191 train_loss= 0.38871 time= 0.00500
Epoch: 0192 train_loss= 0.38679 time= 0.00400
Epoch: 0193 train_loss= 0.38273 time= 0.00436
Epoch: 0194 train_loss= 0.38429 time= 0.00364
Epoch: 0195 train_loss= 0.38483 time= 0.00300
Epoch: 0196 train_loss= 0.38367 time= 0.00303
Epoch: 0197 train_loss= 0.38315 time= 0.00400
Epoch: 0198 train_loss= 0.38490 time= 0.00300
Epoch: 0199 train_loss= 0.38586 ti

Epoch: 0180 train_loss= 0.38770 time= 0.00399
Epoch: 0181 train_loss= 0.38865 time= 0.00400
Epoch: 0182 train_loss= 0.38792 time= 0.00500
Epoch: 0183 train_loss= 0.38649 time= 0.00500
Epoch: 0184 train_loss= 0.38697 time= 0.00400
Epoch: 0185 train_loss= 0.39067 time= 0.00500
Epoch: 0186 train_loss= 0.38914 time= 0.00500
Epoch: 0187 train_loss= 0.38808 time= 0.00400
Epoch: 0188 train_loss= 0.38599 time= 0.00300
Epoch: 0189 train_loss= 0.38736 time= 0.00300
Epoch: 0190 train_loss= 0.38886 time= 0.00300
Epoch: 0191 train_loss= 0.38668 time= 0.00400
Epoch: 0192 train_loss= 0.38550 time= 0.00400
Epoch: 0193 train_loss= 0.38480 time= 0.00300
Epoch: 0194 train_loss= 0.38770 time= 0.00300
Epoch: 0195 train_loss= 0.38917 time= 0.00300
Epoch: 0196 train_loss= 0.38636 time= 0.00400
Epoch: 0197 train_loss= 0.38512 time= 0.00400
Epoch: 0198 train_loss= 0.38593 time= 0.00300
Epoch: 0199 train_loss= 0.38682 time= 0.00400
Epoch: 0200 train_loss= 0.38461 time= 0.00500
Testing model...
Masking test edge

Epoch: 0186 train_loss= 0.39027 time= 0.00400
Epoch: 0187 train_loss= 0.38644 time= 0.00407
Epoch: 0188 train_loss= 0.39114 time= 0.00293
Epoch: 0189 train_loss= 0.38820 time= 0.00300
Epoch: 0190 train_loss= 0.38974 time= 0.00299
Epoch: 0191 train_loss= 0.38810 time= 0.00400
Epoch: 0192 train_loss= 0.38589 time= 0.00400
Epoch: 0193 train_loss= 0.38673 time= 0.00400
Epoch: 0194 train_loss= 0.38922 time= 0.00401
Epoch: 0195 train_loss= 0.39126 time= 0.00399
Epoch: 0196 train_loss= 0.38493 time= 0.00300
Epoch: 0197 train_loss= 0.38850 time= 0.00300
Epoch: 0198 train_loss= 0.38890 time= 0.00400
Epoch: 0199 train_loss= 0.38957 time= 0.00500
Epoch: 0200 train_loss= 0.38964 time= 0.00400
Testing model...
Masking test edges...
Number of bidirectional edges in the graph: 49.0
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 1.55989 time= 0.23095
Epoch: 0002 train_loss= 1.38305 time= 0.00406
Epoch: 0003 train_loss= 0.94769 time= 0.00400
Epoch: 0004 train_loss= 0.74341 time= 

Epoch: 0181 train_loss= 0.39093 time= 0.00381
Epoch: 0182 train_loss= 0.38897 time= 0.00401
Epoch: 0183 train_loss= 0.39047 time= 0.00399
Epoch: 0184 train_loss= 0.39035 time= 0.00399
Epoch: 0185 train_loss= 0.39047 time= 0.00300
Epoch: 0186 train_loss= 0.38819 time= 0.00500
Epoch: 0187 train_loss= 0.39021 time= 0.00500
Epoch: 0188 train_loss= 0.38992 time= 0.00400
Epoch: 0189 train_loss= 0.39127 time= 0.00400
Epoch: 0190 train_loss= 0.38722 time= 0.00299
Epoch: 0191 train_loss= 0.38983 time= 0.00400
Epoch: 0192 train_loss= 0.38932 time= 0.00400
Epoch: 0193 train_loss= 0.38966 time= 0.00387
Epoch: 0194 train_loss= 0.38876 time= 0.00400
Epoch: 0195 train_loss= 0.38921 time= 0.00400
Epoch: 0196 train_loss= 0.38903 time= 0.00301
Epoch: 0197 train_loss= 0.38925 time= 0.00399
Epoch: 0198 train_loss= 0.38762 time= 0.00300
Epoch: 0199 train_loss= 0.38879 time= 0.00500
Epoch: 0200 train_loss= 0.38568 time= 0.00300
Testing model...
Masking test edges...
Number of bidirectional edges in the grap

...

Epoch: 0174 train_loss= 0.39233 time= 0.00501
Epoch: 0175 train_loss= 0.39134 time= 0.00299
Epoch: 0176 train_loss= 0.39520 time= 0.00309
Epoch: 0177 train_loss= 0.39274 time= 0.00406
Epoch: 0178 train_loss= 0.39106 time= 0.00400
Epoch: 0179 train_loss= 0.39377 time= 0.00400
Epoch: 0180 train_loss= 0.39716 time= 0.00400
Epoch: 0181 train_loss= 0.39294 time= 0.00312
Epoch: 0182 train_loss= 0.39761 time= 0.00400
Epoch: 0183 train_loss= 0.39162 time= 0.00400
Epoch: 0184 train_loss= 0.39396 time= 0.00300
Epoch: 0185 train_loss= 0.39364 time= 0.00300
Epoch: 0186 train_loss= 0.39247 time= 0.00406
Epoch: 0187 train_loss= 0.39383 time= 0.00539
Epoch: 0188 train_loss= 0.39606 time= 0.00306
Epoch: 0189 train_loss= 0.39206 time= 0.00401
Epoch: 0190 train_loss= 0.39403 time= 0.00299
Epoch: 0191 train_loss= 0.39089 time= 0.00400
Epoch: 0192 train_loss= 0.39185 time= 0.00300
Epoch: 0193 train_loss= 0.39084 time= 0.00511
Epoch: 0194 train_loss= 0.38959 time= 0.00301
Epoch: 0195 train_loss= 0.39332 ti

Epoch: 0176 train_loss= 0.38886 time= 0.00500
Epoch: 0177 train_loss= 0.38645 time= 0.00507
Epoch: 0178 train_loss= 0.39153 time= 0.00400
Epoch: 0179 train_loss= 0.38875 time= 0.00300
Epoch: 0180 train_loss= 0.38756 time= 0.00500
Epoch: 0181 train_loss= 0.38838 time= 0.00400
Epoch: 0182 train_loss= 0.38855 time= 0.00400
Epoch: 0183 train_loss= 0.38914 time= 0.00478
Epoch: 0184 train_loss= 0.38853 time= 0.00399
Epoch: 0185 train_loss= 0.38749 time= 0.00313
Epoch: 0186 train_loss= 0.38568 time= 0.00400
Epoch: 0187 train_loss= 0.38772 time= 0.00300
Epoch: 0188 train_loss= 0.38793 time= 0.00409
Epoch: 0189 train_loss= 0.38937 time= 0.00500
Epoch: 0190 train_loss= 0.38657 time= 0.00300
Epoch: 0191 train_loss= 0.38918 time= 0.00300
Epoch: 0192 train_loss= 0.38844 time= 0.00411
Epoch: 0193 train_loss= 0.38373 time= 0.00300
Epoch: 0194 train_loss= 0.38900 time= 0.00410
Epoch: 0195 train_loss= 0.38735 time= 0.00402
Epoch: 0196 train_loss= 0.38546 time= 0.00400
Epoch: 0197 train_loss= 0.38908 ti

Epoch: 0170 train_loss= 0.38713 time= 0.00400
Epoch: 0171 train_loss= 0.38924 time= 0.00406
Epoch: 0172 train_loss= 0.38558 time= 0.00400
Epoch: 0173 train_loss= 0.38530 time= 0.00400
Epoch: 0174 train_loss= 0.38372 time= 0.00401
Epoch: 0175 train_loss= 0.38449 time= 0.00399
Epoch: 0176 train_loss= 0.38855 time= 0.00400
Epoch: 0177 train_loss= 0.38646 time= 0.00500
Epoch: 0178 train_loss= 0.38512 time= 0.00500
Epoch: 0179 train_loss= 0.38572 time= 0.00406
Epoch: 0180 train_loss= 0.38573 time= 0.00443
Epoch: 0181 train_loss= 0.38106 time= 0.00405
Epoch: 0182 train_loss= 0.38251 time= 0.00500
Epoch: 0183 train_loss= 0.38226 time= 0.00400
Epoch: 0184 train_loss= 0.38728 time= 0.00403
Epoch: 0185 train_loss= 0.38999 time= 0.00371
Epoch: 0186 train_loss= 0.38632 time= 0.00402
Epoch: 0187 train_loss= 0.38892 time= 0.00400
Epoch: 0188 train_loss= 0.38899 time= 0.00501
Epoch: 0189 train_loss= 0.38846 time= 0.00412
Epoch: 0190 train_loss= 0.38728 time= 0.00404
Epoch: 0191 train_loss= 0.38964 ti

Epoch: 0172 train_loss= 0.39199 time= 0.00408
Epoch: 0173 train_loss= 0.39316 time= 0.00400
Epoch: 0174 train_loss= 0.39389 time= 0.00420
Epoch: 0175 train_loss= 0.39326 time= 0.00409
Epoch: 0176 train_loss= 0.39107 time= 0.00400
Epoch: 0177 train_loss= 0.39265 time= 0.00400
Epoch: 0178 train_loss= 0.39171 time= 0.00400
Epoch: 0179 train_loss= 0.39172 time= 0.00306
Epoch: 0180 train_loss= 0.39290 time= 0.00400
Epoch: 0181 train_loss= 0.39344 time= 0.00400
Epoch: 0182 train_loss= 0.39094 time= 0.00400
Epoch: 0183 train_loss= 0.38997 time= 0.00300
Epoch: 0184 train_loss= 0.39078 time= 0.00505
Epoch: 0185 train_loss= 0.39419 time= 0.00404
Epoch: 0186 train_loss= 0.39302 time= 0.00500
Epoch: 0187 train_loss= 0.38909 time= 0.00499
Epoch: 0188 train_loss= 0.39191 time= 0.00400
Epoch: 0189 train_loss= 0.39347 time= 0.00524
Epoch: 0190 train_loss= 0.38914 time= 0.00479
Epoch: 0191 train_loss= 0.38822 time= 0.00398
Epoch: 0192 train_loss= 0.39136 time= 0.00499
Epoch: 0193 train_loss= 0.39139 ti

Epoch: 0169 train_loss= 0.38776 time= 0.00400
Epoch: 0170 train_loss= 0.38706 time= 0.00407
Epoch: 0171 train_loss= 0.38946 time= 0.00399
Epoch: 0172 train_loss= 0.38672 time= 0.00400
Epoch: 0173 train_loss= 0.39028 time= 0.00404
Epoch: 0174 train_loss= 0.38811 time= 0.00301
Epoch: 0175 train_loss= 0.38833 time= 0.00400
Epoch: 0176 train_loss= 0.38837 time= 0.00300
Epoch: 0177 train_loss= 0.38757 time= 0.00400
Epoch: 0178 train_loss= 0.38974 time= 0.00300
Epoch: 0179 train_loss= 0.39168 time= 0.00400
Epoch: 0180 train_loss= 0.38832 time= 0.00500
Epoch: 0181 train_loss= 0.38902 time= 0.00400
Epoch: 0182 train_loss= 0.38589 time= 0.00414
Epoch: 0183 train_loss= 0.38796 time= 0.00400
Epoch: 0184 train_loss= 0.38693 time= 0.00300
Epoch: 0185 train_loss= 0.38695 time= 0.00417
Epoch: 0186 train_loss= 0.38512 time= 0.00400
Epoch: 0187 train_loss= 0.38797 time= 0.00401
Epoch: 0188 train_loss= 0.38920 time= 0.00517
Epoch: 0189 train_loss= 0.38755 time= 0.00400
Epoch: 0190 train_loss= 0.39069 ti